In [2]:
import pandas as pd
import numpy as np
import time
import yaml
import math
from collections import defaultdict
import itertools
from datetime import datetime, timedelta
import datetime as dt
import random
from scipy import stats
import pickle
from sql_helper import SQLHelper
import re

pd.options.display.float_format = "{:.2f}".format
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [ ]:
# def get_mssql_cursor():
#     config_file_name = 'config.yaml'
#     with open(config_file_name) as config_file:
#         cfg = yaml.load(config_file)
#     sqlhelper = SQLHelper(cfg['sql']['intern_db'], 'mssql')
#     cursor = sqlhelper.get_connection()
#     return cursor
# cursor = get_mssql_cursor()

import pymssql
cursor = pymssql.connect(server='ec2-15-206-55-153.ap-south-1.compute.amazonaws.com', user='internUser', password='WinZO@123', database='master')

In [ ]:
blocked_users_query = """
SELECT TRID,REGISTRATIONID,BLOCK_REASON ,REASON_TEXT ,CREATED_DATE
from TicTokdb.dbo.STBL_USER_BLOCK sub 
where REASON_TEXT IN ('Callbreak Collusion',
'Callbreak Collusion - Akash',
'Callbreak Collusion - Blocked by Akash',
'CallBreak Collusion Fraud',
'CallBreak Collusion Fraud - by Susanta do not unblock',
'Callbreak Collusion- Akash',
'Callbreak Collusion: Akash', 'Poker Hacking - Dhiraj','Poker Chip Dumping June End','Poker Fraud', 'Poker Fraud Dhiraj')
"""
blocked_users_df = pd.read_sql(blocked_users_query, cursor)
blocked_users_df.to_pickle('DataBase/Block_user.pkl')
blocked_users_df.head(2)

# Blocked User

In [3]:
blocked_users_dfff= pd.read_pickle('DataBase/Block_user.pkl')
blocked_users_list = blocked_users_dfff['REGISTRATIONID'].unique().tolist()
len(blocked_users_list)

2071

### Labeled Data

In [4]:
Labeled_data = pd.read_csv('Callbreak colluders list.csv')
Labeled_ID_list = Labeled_data.P1_regID.unique().tolist()
Labeled_data.shape

(412, 2)

In [ ]:
lst = (set(blocked_users_list)).union(set(Labeled_ID_list))
lst = list(lst)
print(len(lst))

In [ ]:
Labeled_ID_sublist_str = ','.join(str(u) for u in Labeled_ID_list)
blocked_users_query = """
SELECT TRID,REGISTRATIONID,BLOCK_REASON ,REASON_TEXT ,CREATED_DATE
FROM TicTokdb.dbo.STBL_USER_BLOCK
where REGISTRATIONID in ({})
""".format(Labeled_ID_sublist_str)
blocked_users_df_temp = pd.read_sql(blocked_users_query, cursor)
blocked_users_df = pd.concat([blocked_users_df, blocked_users_df_temp])

blocked_users_df.to_pickle('DataBase/Block_user&Labeled_df.pkl')


In [ ]:
%%time
i = 0
user_reg_df = pd.DataFrame()
user_loc_df = pd.DataFrame()
user_contacts_df = pd.DataFrame()
user_apps_df = pd.DataFrame()
user_Rake_df = pd.DataFrame()
Add_Cash_df = pd.DataFrame()
user_commongames_df = pd.DataFrame()

while i <= len(lst):
    print(i)
    blocked_users_sublist = lst[i:i+500]
    blocked_users_sublist_str = ','.join(str(u) for u in blocked_users_sublist)

    
    user_reg_query = """
    SELECT REGISTRATIONID,NAME,REGISTRATION_DATE,DEVICE_INFO,STATUS from MongoDB.dbo.stbl_Registration
    where REGISTRATIONID in ({})
    """.format(blocked_users_sublist_str)
    user_reg_df_temp = pd.read_sql(user_reg_query, cursor)
    user_reg_df = pd.concat([user_reg_df, user_reg_df_temp])
    

    user_loc_query = """
    SELECT REGISTRATIONID,LATITUDE,LONGITUDE from TicTokdb.dbo.STBL_USER_LOCATION
    where REGISTRATIONID in ({})
    """.format(blocked_users_sublist_str)
    user_loc_df_temp = pd.read_sql(user_loc_query, cursor)
    user_loc_df = pd.concat([user_loc_df, user_loc_df_temp])

    user_contacts_query = """
    select TRID,REGISTRATIONID,NAME,CREATED_DATE from MongoDB.DBO.STBL_PHONE_CONTACTS 
    where REGISTRATIONID in ({})
    """.format(blocked_users_sublist_str)
    #     print(user_contacts_query)
    user_contacts_df_temp = pd.read_sql(user_contacts_query, cursor)
    user_contacts_df = pd.concat([user_contacts_df, user_contacts_df_temp])

    user_apps_query = """
    SELECT REGISTRATIONID,APP_NAME_ID from TicTokdb.dbo.STBL_USER_APP_NAMES
    where REGISTRATIONID in ({})
    """.format(blocked_users_sublist_str)
    #     print(user_apps_query)
    user_apps_df_temp = pd.read_sql(user_apps_query, cursor)
    user_apps_df = pd.concat([user_apps_df, user_apps_df_temp])

    user_Rake_query = """
    SELECT CATEGORY,REGISTRATIONID,RAKE from MongoDB.analysis.DAILY_FORMATWISE_RAKE
    where REGISTRATIONID in ({})
    """.format(blocked_users_sublist_str)
    user_Rake_df_temp = pd.read_sql(user_Rake_query, cursor)
    user_Rake_df = pd.concat([user_Rake_df, user_Rake_df_temp])
    

    Add_Cash_query = """
    SELECT REGISTRATIONID,PAYAMOUNT,PAYDATE,PAYSTATUS,CREATED_DATE FROM TicTokdb.dbo.STBL_CASH_IN_WALLET
    where REGISTRATIONID in ({}) AND PAYSTATUS = 'SUCCESS'
    """.format(blocked_users_sublist_str)
    Add_Cash_df_temp = pd.read_sql(Add_Cash_query, cursor)
    Add_Cash_df = pd.concat([Add_Cash_df, Add_Cash_df_temp])
    
    wb_bazzi_trans_query = """
    SELECT REGISTRATIONID,CHALLENGE_ID,AMOUNT,TRANS_TYPE FROM TicTokdb.dbo.STBL_WB_BAAZI_TRANSACTION swbt
    where REGISTRATIONID in ({})
    """.format(blocked_users_sublist_str)
    user_commongames_df_temp = pd.read_sql(wb_bazzi_trans_query, cursor)
    user_commongames_df = pd.concat([user_commongames_df, user_commongames_df_temp])

    
    i += 500
    

# print(user_loc_df.shape)
Add_Cash_df = Add_Cash_df.loc[Add_Cash_df.groupby('REGISTRATIONID').PAYDATE.idxmin()].reset_index(drop=True)
user_Rake_df = user_Rake_df.loc[user_Rake_df.groupby('REGISTRATIONID').RAKE.idxmax()].reset_index(drop=True)

# Add_Cash_df['PAYDATE'] = pd.to_datetime(Add_Cash_df['PAYDATE'].fillna('1990-09-18 19:56:26.267'))
Add_Cash_df = Add_Cash_df.loc[Add_Cash_df.groupby('REGISTRATIONID').PAYDATE.idxmin()].reset_index(drop=True)
user_Rake_df = user_Rake_df.loc[user_Rake_df.groupby('REGISTRATIONID').RAKE.idxmax()].reset_index(drop=True)

### Save Blocked_users_details into pickle

In [ ]:
# user_reg_df.to_pickle('DataBase/reg.pkl')
# user_loc_df.to_pickle('DataBase/loc.pkl')
# user_contacts_df.to_pickle('DataBase/contact.pkl')
# user_apps_df.to_pickle('DataBase/apps.pkl')
# user_Rake_df.to_pickle('DataBase/max_Rake.pkl')
# Add_Cash_df.to_pickle('DataBase/First_AddCash.pkl')
# user_commongames_df.to_pickle('DataBase/user_commongames.pkl')
# blocked_users_df.to_pickle('DataBase/Block_user&Labeled_df.pkl')

# Load All Pickle File

In [ ]:
user_reg_df= pd.read_pickle('DataBase/reg.pkl')
user_loc_df=pd.read_pickle('DataBase/loc.pkl')
user_contacts_df=pd.read_pickle('DataBase/contact.pkl')
user_apps_df=pd.read_pickle('DataBase/apps.pkl')
user_Rake_df=pd.read_pickle('DataBase/max_Rake.pkl')
Add_Cash_df= pd.read_pickle('DataBase/First_AddCash.pkl')
user_commongames_df=pd.read_pickle('DataBase/user_commongames.pkl')
blocked_users_df= pd.read_pickle('DataBase/Block_user&Labeled_df.pkl')

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def get_distance_between_locs(p1_lat, p1_lon, p2_lat, p2_lon):
    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(p1_lat)
    lon1 = radians(p1_lon)
    lat2 = radians(p2_lat)
    lon2 = radians(p2_lon)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c 
    return distance
# Manufacture details handled inphone , separated by "," , separated by " "
def get_MANUFACTURER(device_info):
    S =  'Device Name'
    T = 'MANUFACTURER'
    data = [k for k in device_info.split(':')]
    for t in range(len(data)):
        if (T in data[t]) or (S in data[t]):
            s = re.split(' Device OS| BRAND',data[t+1])[0]
            s = s.split(',',1)[0]
            if s[0] == ' ' or s[-1]== ' ':
                s = s.replace(' ','',1)
            return s
def get_MODEL(device_info):
    S = 'Device Modal'
    T = 'MODEL'
    data = [k for k in device_info.split(':')]
    for t in range(len(data)):
        if (T in data[t]) or (S in data[t]):
            s = re.split(' Device OS| BRAND',data[t+1])[0]
            s = s.split(',',1)[0]
            if s[0] == ' ' or s[-1]== ' ':
                s = s.replace(' ','',1)
            return s
def contact_overlap(p1,p2):
    p1_ = set(p1) 
    p2_ = set(p2)
    if len(p1_.union(p2_)) == 0:
        k = 0
    else:
        k = ( len(p1_.intersection(p2_))/len(p1_.union(p2_)) ) * 100
    return k
def app_overlap(p1,p2):
    p1_ = set(p1) 
    p2_ = set(p2)
    if len(p1_.union(p2_)) ==0:
        k=0
    else:
        k =( len(p1_.intersection(p2_))/len(p1_.union(p2_)) ) * 100
    return k
def levenshtein_distance(a,b):
    n, m = len(a), len(b)
    if n > m:
        # Make sure n <= m, to use O(min(n,m)) space
        a,b = b,a
        n,m = m,n

    current = range(n+1)
    for i in range(1,m+1):
        previous, current = current, [i]+[0]*n
        for j in range(1,n+1):
            add, delete = previous[j]+1, current[j-1]+1
            change = previous[j-1]
            if a[j-1] != b[i-1]:
                change = change + 1
            current[j] = min(add, delete, change)
    return current[n]
def similarity(A,B):    
    if __name__=="__main__":
        
        k = levenshtein_distance(A,B)  #  k is the similarity coeficients between these two strings A,B
        if (len(A)+len(B))==0:
            k_normalise = np.nan
        else:
            k_normalise = k/(len(A)+len(B))
        return k_normalise

In [ ]:
user_reg_df.set_index(['REGISTRATIONID'],inplace=True)
user_loc_df.set_index(['REGISTRATIONID'],inplace=True)
user_contacts_df.set_index(['REGISTRATIONID'],inplace=True)
user_apps_df.set_index(['REGISTRATIONID'],inplace=True)
Add_Cash_df.set_index(['REGISTRATIONID'],inplace=True)
user_Rake_df.set_index(['REGISTRATIONID'],inplace=True)
blocked_users_df.set_index(['REGISTRATIONID'],inplace=True)
user_commongames_df.set_index(['REGISTRATIONID'],inplace=True)


credit_df = user_commongames_df[user_commongames_df.TRANS_TYPE == 'C']
debit_df  = user_commongames_df[user_commongames_df.TRANS_TYPE == 'D']

In [ ]:
%%time
f_loc_time = []
f_block_reason_time = []
f_block_time = []
f_Rake_time = []
frst_addcash_time = []
f_name_time = []
f_device_time = []
model_manufact_time = []
contact_time = []
f_app = []
f_game_time = []
fn_dist_time = []
f_reg_time = []
Name_sim_time = []
contact_overlap_time = []
app_match_time = []
f_comgame_time = []
comgame_percent_of_A_B_time = []
A_B_profit_timand_C_D_time = []


user_already_processed = set()
block_df_list = []
count = 0

for user in blocked_users_list:
    
    if count%300 == 0:
        print(user,count)
    count+=1
    
    start = time.time_ns()
    p1_location = user_loc_df[user_loc_df.index==user]
    if p1_location.shape[0] > 0:
        p1_lat, p1_lon = p1_location['LATITUDE'].iloc[0], p1_location['LONGITUDE'].iloc[0]
    else:
        p1_lat, p1_lon = 0, 0
    end = time.time_ns()
    f_loc_time.append(end-start)

#     s = time.time_ns()
#     try:
#         p1_block_reason = blocked_users_df[blocked_users_df.index==user]['REASON_TEXT'].iloc[0]
#     except:
#         p1_block_reason = ''
#     e = time.time_ns()
#     f_block_reason_time.append(e-s)

    s = time.time_ns()
    try:
        p1_block_time = blocked_users_df[blocked_users_df.index==user]['CREATED_DATE'].iloc[0]
    except:
        p1_block_time = ''
    e = time.time_ns()
    f_block_time.append(e-s)
    #    #    #
    
    s = time.time_ns()
    try:
        Rake = user_Rake_df[user_Rake_df.index==user]
        Rake_value_p1 = Rake['RAKE'].iloc[0]
        Category_p1 = Rake['CATEGORY'].iloc[0]
    except:
        Rake_value_p1 = np.nan
        Category_p1 =''
    e = time.time_ns()
    f_Rake_time.append(e-s)

    s = time.time_ns()    
    try:
        Frst_AddCash_p1 = Add_Cash_df[Add_Cash_df.index==user]['PAYAMOUNT'].iloc[0]
    except:
        Frst_AddCash_p1  = np.nan
#     try:
#         Frst_AddCash_Time_p1  = Add_Cash_df[Add_Cash_df.index==user]['PAYDATE'].iloc[0]
#     except:
#         Frst_AddCash_Time_p1  = '1990-09-18 19:56:26.267'
    e = time.time_ns()
    frst_addcash_time.append(e-s)

    s = time.time_ns()
    try:
        p1_name   = user_reg_df[user_reg_df.index==user]['NAME'].iloc[0]
    except:
        p1_name = ''
    e = time.time_ns()
    f_name_time.append(e-s)
    #    #    #
    
#     s = time.time_ns()
#     try:   
#         p1_device = user_reg_df[user_reg_df.index==user]['DEVICE_INFO'].iloc[0]
#     except:
#         p1_device = ''
#     e = time.time_ns()
#     f_device_time.append(e-s)

#     s = time.time_ns()
#     p1_model    = get_MODEL(p1_device)
#     p1_manufact = get_MANUFACTURER(p1_device)
#     e = time.time_ns()
#     model_manufact_time.append(e-s)

    s = time.time_ns()
    p1_contact  =  user_contacts_df[user_contacts_df.index==user]['NAME']
    e = time.time_ns()
    contact_time.append(e-s)

    s = time.time_ns()
    p1_app     = user_apps_df[user_apps_df.index==user]['APP_NAME_ID']
    e = time.time_ns()
    f_app.append(e-s)
    #    #    #
    s = time.time_ns()
    p1_game = set(user_commongames_df[user_commongames_df.index==user]['CHALLENGE_ID'])
    e = time.time_ns()
    f_game_time.append(e-s)
    
#################    
    for user2 in blocked_users_list:
        if ((user2 not in user_already_processed)&(user != user2)):
                        
            p2_location = user_loc_df[user_loc_df.index==user2]
            if p2_location.shape[0] > 0:
                p2_lat, p2_lon = p2_location['LATITUDE'].iloc[0], p2_location['LONGITUDE'].iloc[0]
            else:
                p2_lat, p2_lon = 0, 0 

            p1_p2_distance = np.nan
            if p1_location.shape[0] > 0 and p2_location.shape[0] > 0:
                begin = time.time_ns()
                p1_p2_distance = get_distance_between_locs(p1_lat, p1_lon, p2_lat, p2_lon)   ###
                end = time.time_ns()
                fn_dist_time.append(end-begin)
#################            
            if p1_p2_distance < 100:
#                 try:
#                     p2_block_reason = blocked_users_df[blocked_users_df.index==user2]['REASON_TEXT'].iloc[0]
#                 except:
#                     p2_block_reason = ''

                begin = time.time_ns()
                try:
                    p2_reg_time = user_reg_df[user_reg_df.index==user2]['REGISTRATION_DATE'].iloc[0]
                except:
                    p2_reg_time = ''
                end = time.time_ns()
                f_reg_time.append(end-begin)
#                 (s2-s1).total_seconds()/60
                try:
                    Time_diff_p1block_p2reg_minutes = (p2_reg_time - p1_block_time).total_seconds()/60
                except:
                    Time_diff_p1block_p2reg_minutes = np.nan

                try:
                    Rake2 = user_Rake_df[user_Rake_df.index==user2]
                    Rake_value2 = Rake2['RAKE'].iloc[0]
                    Category2 = Rake2['CATEGORY'].iloc[0]            
                except:
                    Rake_value2 = np.nan
                    Category2 =''

                try:
                    Frst_AddCash2 = Add_Cash_df[Add_Cash_df.index==user2]['PAYAMOUNT'].iloc[0]
#                     Frst_AddCash_Time2  = Add_Cash_df[Add_Cash_df.index==user2]['PAYDATE'].iloc[0]
                except:
                    Frst_AddCash2  = np.nan
#                     Frst_AddCash_Time2  = '1990-09-18 19:56:26.267'

                try:
                    p2_name   = user_reg_df[user_reg_df.index==user2]['NAME'].iloc[0]
                except:
                    p2_name = ''

                beg = time.time_ns() 
                name_similarity = similarity(p1_name,p2_name)
                end = time.time_ns()
                Name_sim_time.append(end-beg)


#                 try:
#                     p2_device = user_reg_df[user_reg_df.index==user2]['DEVICE_INFO'].iloc[0]
#                 except:
#                     p2_device = ''

#                 p2_model    = get_MODEL(p2_device)
#                 p2_manufact = get_MANUFACTURER(p2_device)

                p2_contact  =  user_contacts_df[user_contacts_df.index==user2]['NAME']

                beg = time.time_ns()
                matching_contact_count = contact_overlap(p1_contact,p2_contact)
                end = time.time_ns()
                contact_overlap_time.append(end-beg)

                p2_app     = user_apps_df[user_apps_df.index==user2]['APP_NAME_ID']

                beg = time.time_ns()
                app_match  = app_overlap(p1_app,p2_app)
                end = time.time_ns()
                app_match_time.append(end-beg)

                p2_game = set(user_commongames_df[user_commongames_df.index==user2]['CHALLENGE_ID'])
                # common_games_A_B

                beg = time.time_ns()
                common_games = p1_game.intersection(p2_game)
                end = time.time_ns()
                f_comgame_time.append(end-beg)
                
                if len(common_games) != 0 :

                    beg = time.time_ns()
                    # common_games_A_B_percent_of_A%B_games
                    common_games_A_B_percent_of_A_games = (len(common_games)/len(p1_game) )*100                    
                    common_games_A_B_percent_of_B_games = (len(common_games)/len(p2_game) )*100
                    end = time.time_ns()
                    comgame_percent_of_A_B_time.append(end-beg)

                    beg = time.time_ns()
                    # common_games_A_B_profit   && # common_games_A_B_c_by_d
                    
                    com_game_credit_A = credit_df[(credit_df.index == user) & (credit_df.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()
                    com_game_debit_A  = debit_df[(debit_df.index == user) & (debit_df.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()
                    com_game_credit_B = credit_df[(credit_df.index == user2) & (credit_df.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()
                    com_game_debit_B  = debit_df[(debit_df.index == user2) & (debit_df.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()

                    if ((com_game_debit_A+com_game_debit_B) != 0):
                        A_B_Profit = ((com_game_credit_A+com_game_credit_B) - (com_game_debit_A+com_game_debit_B))/(com_game_debit_A+com_game_debit_B)
                        A_B_C_BY_D = (com_game_credit_A+com_game_credit_B)/(com_game_debit_A+com_game_debit_B)
                    else:
                        A_B_Profit =  np.nan
                        A_B_C_BY_D =  np.nan


                    end = time.time_ns()
                    A_B_profit_timand_C_D_time.append(end-beg)


                    
                    block_df_list.append({
                            'P1_regID':user,
                            'P2_regID': user2,
                            'p1_name':p1_name,
                            'p2_name':p2_name,
#                             'p1_block_reason' : p1_block_reason,
#                             'p2_block_reason' : p2_block_reason,
#                             'p1_block_time' : p1_block_time,
#                             'p2_reg_time'   : p2_reg_time,
                            'Diff_p1block_p2_reg_minutes' : Time_diff_p1block_p2reg_minutes,
                            'Name_similarity': name_similarity,
                            'Contacts_Overlap_%': matching_contact_count,
                            'p1_p2_distance_KM' : p1_p2_distance,
                            'Apps_overlap_%': app_match,
#                             'p1_model': p1_model,
#                             'p2_model': p2_model,
#                             'p1_manufact': p1_manufact,
#                             'p2_manufact': p2_manufact,

                            'Rake' : Rake_value_p1,
                            'Category' : Category_p1,
                            'Rake2' : Rake_value2,
                            'Category2' : Category2,
                            'Frst_AddCash': Frst_AddCash_p1,
                            'Frst_AddCash2': Frst_AddCash2,
#                             'Frst_AddCash_Time':Frst_AddCash_Time_p1,
#                             'Frst_AddCash_Time2': Frst_AddCash_Time2,
                            'common_games' : list(common_games),
                            'common_gameAB_%_of_A' : common_games_A_B_percent_of_A_games,
                            'common_gameAB_%_of_B' : common_games_A_B_percent_of_B_games,
                            'A_B_Profit':A_B_Profit,
                            'A_B_C_BY_D':A_B_C_BY_D,
                            

                        })
                else:
                    block_df_list.append({
                        'P1_regID':user,
                        'P2_regID': user2,
                        'p1_name':p1_name,
                        'p2_name':p2_name,
#                         'p1_block_reason' : p1_block_reason,
#                         'p2_block_reason' : p2_block_reason,
#                         'p1_block_time' : p1_block_time,
#                         'p2_reg_time'   : p2_reg_time,
                        'Diff_p1block_p2_reg_minutes' : Time_diff_p1block_p2reg_minutes,
                        'Name_similarity': name_similarity,
                        'Contacts_Overlap_%': matching_contact_count,
                        'p1_p2_distance_KM' : p1_p2_distance,
                        'Apps_overlap_%': app_match,
#                         'p1_model': p1_model,
#                         'p2_model': p2_model,
#                         'p1_manufact': p1_manufact,
#                         'p2_manufact': p2_manufact,

                        'Rake' : Rake_value_p1,
                        'Category' : Category_p1,
                        'Rake2' : Rake_value2,
                        'Category2' : Category2,
                        'Frst_AddCash': Frst_AddCash_p1,
                        'Frst_AddCash2': Frst_AddCash2,
#                         'Frst_AddCash_Time':Frst_AddCash_Time_p1,
#                         'Frst_AddCash_Time2': Frst_AddCash_Time2
                    })
    
    user_already_processed.add(user)
                        
#     print('About to Complete')

blocked_details_df = pd.DataFrame(block_df_list)
blocked_details_df.to_pickle('blocked_details_df.pkl')
blocked_details_df.tail()


### Time of each operation

In [ ]:
f_loc_time_mean = np.mean(f_loc_time)
f_block_reason_time_mean = np.mean(f_block_reason_time)
f_block_time_mean = np.mean(f_block_time)
f_Rake_time_mean = np.mean(f_Rake_time)
frst_addcash_time_mean = np.mean(frst_addcash_time)
f_name_time_mean = np.mean(f_name_time)
f_device_time_mean = np.mean(f_device_time)
model_manufact_time_mean = np.mean(model_manufact_time)
contact_time_mean = np.mean(contact_time)
f_app_mean = np.mean(f_app)
f_game_time_mean = np.mean(f_game_time)
fn_dist_time_mean = np.mean(fn_dist_time)
f_reg_time_mean = np.mean(f_reg_time)
Name_sim_time_mean = np.mean(Name_sim_time)
contact_overlap_time_mean = np.mean(contact_overlap_time)
app_match_time_mean = np.mean(app_match_time)
f_comgame_time_mean = np.mean(f_comgame_time)
comgame_percent_of_A_B_time_mean = np.mean(comgame_percent_of_A_B_time)
A_B_profit_timand_C_D_time_mean = np.mean(A_B_profit_timand_C_D_time)

Column  = ['f_loc_time','f_block_reason_time','f_block_time','f_Rake_time','frst_addcash_time','f_name_time',
             'f_device_time','model_manufact_time','contact_time','f_app','f_game_time','fn_dist_time','f_reg_time',
             'Name_sim_time','contact_overlap_time','app_match_time','f_comgame_time','comgame_percent_of_A_B_time',
             'A_B_profit_timand_C_D_time']

Time_list = [f_loc_time_mean,f_block_reason_time_mean,f_block_time_mean,f_Rake_time_mean,frst_addcash_time_mean,f_name_time_mean,
             f_device_time_mean,model_manufact_time_mean,contact_time_mean,f_app_mean,f_game_time_mean,fn_dist_time_mean,f_reg_time_mean,
             Name_sim_time_mean,contact_overlap_time_mean,app_match_time_mean,f_comgame_time_mean,comgame_percent_of_A_B_time_mean,
             A_B_profit_timand_C_D_time_mean]


Time_df = pd.DataFrame(list(zip(Column, Time_list)),
               columns =['Name', 'val'])
Time_df.dropna(axis=0,inplace=True)
Time_df.to_pickle('DataBase/Time_ns_df_some_users.pkl')
Time_df.sort_values('val',ascending =False)

## Features_For_Labeled_Data

In [ ]:
%%time

def get_Labeldata_features(Labeled_data,blocked_users_df,user_reg_df,user_loc_df,user_contacts_df,user_apps_df,user_Rake_df,Add_Cash_df,user_commongames_df):
    
    Labeled_p1 = Labeled_data.P1_regID.tolist()
    Labeled_p2 = Labeled_data.P2_regID.tolist()
    # user_already_processed = set()
    labeled_data_list = []
    count = 0

    for i in range(len(Labeled_p1)):
        user  = Labeled_p1[i]
        user2 = Labeled_p2[i]
        if count%100 == 0:
            print(user,count)
        count+=1

        p1_location = user_loc_df[user_loc_df.index==user]
        if p1_location.shape[0] > 0:
            p1_lat, p1_lon = p1_location['LATITUDE'].iloc[0], p1_location['LONGITUDE'].iloc[0]
        else:
            p1_lat, p1_lon = 0, 0

    #     try:
    #         p1_block_reason = blocked_users_df[blocked_users_df.index==user]['REASON_TEXT'].iloc[0]
    #     except:
    #         p1_block_reason = ''

        try:
            p1_block_time = blocked_users_df[blocked_users_df.index==user]['CREATED_DATE'].iloc[0]
        except:
            p1_block_time = ''
        #    #    # 
        try:
            Rake = user_Rake_df[user_Rake_df.index==user]
            Rake_value_p1 = Rake['RAKE'].iloc[0]
            Category_p1 = Rake['CATEGORY'].iloc[0]
        except:
            Rake_value_p1 = np.nan
            Category_p1 =''

        try:
            Frst_AddCash_p1 = Add_Cash_df[Add_Cash_df.index==user]['PAYAMOUNT'].iloc[0]
        except:
            Frst_AddCash_p1  = np.nan
    #     try:
    #         Frst_AddCash_Time_p1  = Add_Cash_df[Add_Cash_df.index==user]['PAYDATE'].iloc[0]
    #     except:
    #         Frst_AddCash_Time_p1  = '1990-09-18 19:56:26.267'

        try:
            p1_name   = user_reg_df[user_reg_df.index==user]['NAME'].iloc[0]
        except:
            p1_name = ''

        #    #    #
    #     try:   
    #         p1_device = user_reg_df[user_reg_df.index==user]['DEVICE_INFO'].iloc[0]
    #     except:
    #         p1_device = ''
    #     p1_model    = get_MODEL(p1_device)
    #     p1_manufact = get_MANUFACTURER(p1_device)

        p1_contact  =  user_contacts_df[user_contacts_df.index==user]['NAME']
        p1_app     = user_apps_df[user_apps_df.index==user]['APP_NAME_ID']
        p1_game = set(user_commongames_df[user_commongames_df.index==user]['CHALLENGE_ID'])
     ############################

        p2_location = user_loc_df[user_loc_df.index==user2]
        if p2_location.shape[0] > 0:
            p2_lat, p2_lon = p2_location['LATITUDE'].iloc[0], p2_location['LONGITUDE'].iloc[0]
        else:
            p2_lat, p2_lon = 0, 0 

        p1_p2_distance = np.nan
        if p1_location.shape[0] > 0 and p2_location.shape[0] > 0:
            p1_p2_distance = get_distance_between_locs(p1_lat, p1_lon, p2_lat, p2_lon)   ###


    #                 try:
    #                     p2_block_reason = blocked_users_df[blocked_users_df.index==user2]['REASON_TEXT'].iloc[0]
    #                 except:
    #                     p2_block_reason = ''

        try:
            p2_reg_time = user_reg_df[user_reg_df.index==user2]['REGISTRATION_DATE'].iloc[0]
        except:
            p2_reg_time = ''

        try:
            Time_diff_p1block_p2reg_minutes = (p2_reg_time - p1_block_time).total_seconds()/60
        except:
            Time_diff_p1block_p2reg_minutes = np.nan

        try:
            Rake2 = user_Rake_df[user_Rake_df.index==user2]
            Rake_value2 = Rake2['RAKE'].iloc[0]
            Category2 = Rake2['CATEGORY'].iloc[0]            
        except:
            Rake_value2 = np.nan
            Category2 =''

        try:
            Frst_AddCash2 = Add_Cash_df[Add_Cash_df.index==user2]['PAYAMOUNT'].iloc[0]
    #                     Frst_AddCash_Time2  = Add_Cash_df[Add_Cash_df.index==user2]['PAYDATE'].iloc[0]
        except:
            Frst_AddCash2  = np.nan
    #                     Frst_AddCash_Time2  = '1990-09-18 19:56:26.267'

        try:
            p2_name   = user_reg_df[user_reg_df.index==user2]['NAME'].iloc[0]
        except:
            p2_name = ''
        name_similarity = similarity(p1_name,p2_name)

    #                 try:
    #                     p2_device = user_reg_df[user_reg_df.index==user2]['DEVICE_INFO'].iloc[0]
    #                 except:
    #                     p2_device = ''

    #                 p2_model    = get_MODEL(p2_device)
    #                 p2_manufact = get_MANUFACTURER(p2_device)
        p2_contact  =  user_contacts_df[user_contacts_df.index==user2]['NAME']
        matching_contact_count = contact_overlap(p1_contact,p2_contact)
        p2_app     = user_apps_df[user_apps_df.index==user2]['APP_NAME_ID']
        app_match  = app_overlap(p1_app,p2_app)

        p2_game = set(user_commongames_df[user_commongames_df.index==user2]['CHALLENGE_ID'])
        # common_games_A_B
        common_games = p1_game.intersection(p2_game)

        if len(common_games) != 0 :

            # common_games_A_B_percent_of_A%B_games
            common_games_A_B_percent_of_A_games = (len(common_games)/len(p1_game) )*100                    
            common_games_A_B_percent_of_B_games = ( len(common_games)/len(p2_game) )*100

            # common_games_A_B_profit   && # common_games_A_B_c_by_d

            com_game_credit_A = credit_df[(credit_df.index == user) & (credit_df.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()
            com_game_debit_A  = debit_df[(debit_df.index == user) & (debit_df.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()
            com_game_credit_B = credit_df[(credit_df.index == user2) & (credit_df.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()
            com_game_debit_B  = debit_df[(debit_df.index == user2) & (debit_df.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()

            if ((com_game_debit_A+com_game_debit_B) != 0):
                A_B_Profit = ((com_game_credit_A+com_game_credit_B) - (com_game_debit_A+com_game_debit_B))/(com_game_debit_A+com_game_debit_B)
                A_B_C_BY_D = (com_game_credit_A+com_game_credit_B)/(com_game_debit_A+com_game_debit_B)
            else:
                A_B_Profit =  np.nan
                A_B_C_BY_D =  np.nan

            labeled_data_list.append({
                    'P1_regID':user,
                    'P2_regID': user2,
                    'p1_name':p1_name,
                    'p2_name':p2_name,
    #                             'p1_block_reason' : p1_block_reason,
    #                             'p2_block_reason' : p2_block_reason,
    #                             'p1_block_time' : p1_block_time,
    #                             'p2_reg_time'   : p2_reg_time,
                    'Diff_p1block_p2_reg_minutes' : Time_diff_p1block_p2reg_minutes,
                    'Name_similarity': name_similarity,
                    'Contacts_Overlap_%': matching_contact_count,
                    'p1_p2_distance_KM' : p1_p2_distance,
                    'Apps_overlap_%': app_match,
    #                             'p1_model': p1_model,
    #                             'p2_model': p2_model,
    #                             'p1_manufact': p1_manufact,
    #                             'p2_manufact': p2_manufact,

                    'Rake' : Rake_value_p1,
                    'Category' : Category_p1,
                    'Rake2' : Rake_value2,
                    'Category2' : Category2,
                    'Frst_AddCash': Frst_AddCash_p1,
                    'Frst_AddCash2': Frst_AddCash2,
    #                             'Frst_AddCash_Time':Frst_AddCash_Time_p1,
    #                             'Frst_AddCash_Time2': Frst_AddCash_Time2,
                    'common_games' : list(common_games),
                    'common_gameAB_%_of_A' : common_games_A_B_percent_of_A_games,
                    'common_gameAB_%_of_B' : common_games_A_B_percent_of_B_games,
                    'A_B_Profit':A_B_Profit,
                    'A_B_C_BY_D':A_B_C_BY_D,


                })
        else:
            labeled_date_list.append({
                'P1_regID':user,
                'P2_regID': user2,
                'p1_name':p1_name,
                'p2_name':p2_name,
    #                         'p1_block_reason' : p1_block_reason,
    #                         'p2_block_reason' : p2_block_reason,
    #                         'p1_block_time' : p1_block_time,
    #                         'p2_reg_time'   : p2_reg_time,
                'Diff_p1block_p2_reg_minutes' : Time_diff_p1block_p2reg_minutes,
                'Name_similarity': name_similarity,
                'Contacts_Overlap_%': matching_contact_count,
                'p1_p2_distance_KM' : p1_p2_distance,
                'Apps_overlap_%': app_match,
    #                         'p1_model': p1_model,
    #                         'p2_model': p2_model,
    #                         'p1_manufact': p1_manufact,
    #                         'p2_manufact': p2_manufact,

                'Rake' : Rake_value_p1,
                'Category' : Category_p1,
                'Rake2' : Rake_value2,
                'Category2' : Category2,
                'Frst_AddCash': Frst_AddCash_p1,
                'Frst_AddCash2': Frst_AddCash2,
    #                         'Frst_AddCash_Time':Frst_AddCash_Time_p1,
    #                         'Frst_AddCash_Time2': Frst_AddCash_Time2
            })

    #     print('About to Complete')

    Labeled_user_details_df = pd.DataFrame(labeled_data_list)
    Labeled_user_details_df.to_pickle('Labeled_user_details_df.pkl')
    return Labeled_user_details_df

Labeled_user_details_df = get_Labeldata_features(Labeled_data,blocked_users_df,user_reg_df,user_loc_df,user_contacts_df,user_apps_df,user_Rake_df,Add_Cash_df,user_commongames_df)

In [ ]:
Labeled_user_details_df['Class'] = 1
blocked_details_df['Class'] = 0

In [ ]:
Feature_df = pd.concat([blocked_details_df,Labeled_user_details_df])
Feature_df.to_pickle('Feature_df.pkl')

In [ ]:
Feature_df.head()